## Environmental

python 3.10.14  
pyqubo 

In [1]:
from dwave.system import DWaveSampler, EmbeddingComposite
import numpy as np
from pyqubo import Array, Constraint, Placeholder
import pandas as pd
import os
import config

## Nqueen solving

In [47]:
# Nqueen

# number of queens
N = 8

# Binary variables
# x[i][j] = 1 if a queen is placed at cell (i, j)
x = Array.create('x', shape=(N, N), vartype='BINARY')

# Constraint 1: Each row has exactly one queen
# \sum_{j} x[i][j] = 1 for all i
row_const = 0
for i in range(N):
    row_const += (np.sum(x[i, :]) - 1) ** 2

# Constraint 2: Each column has exactly one queen
# \sum_{i} x[i][j] = 1 for all j
col_const = 0
for j in range(N):
    col_const += (np.sum(x[:, j]) - 1) ** 2

# Constaraint 3 : Each diagonal has at most one queen
diag_const = 0

# left to right diagonal
for i in range(-N+1, N):
    tmp = np.sum(np.diag(x, k=i))
    diag_const += (tmp * (tmp - 1))

# right to left diagonal
for i in range(-N+1, N):
    tmp = np.sum(np.diag(np.fliplr(x), k=i))
    diag_const += (tmp * (tmp - 1))


# Constraint 4: number of queens is N
num_const = 0
tmp = 0
for i in range(N):
    for j in range(N):
        tmp += x[i][j]

num_const += (tmp - N) ** 2

# Define the Hamiltonian
H = Placeholder('lambda_rowcol') * Constraint(row_const, label='row') \
  + Placeholder('lambda_rowcol') * Constraint(col_const, label='col') \
  + Placeholder('lambda_diag')   * Constraint(diag_const, label='diag') \
  + Placeholder('lamda_num')     * Constraint(num_const, label='num')

# Compile the model
model = H.compile()



In [41]:
#parameters
feed_dict = {'lambda_rowcol': 2.0, 'lambda_diag': 2.0, 'lamda_num': 1.0}

# QUBO
qubo, offset = model.to_qubo(feed_dict=feed_dict)

In [42]:
# Set up the D-Wave sampler
endpoint = "https://cloud.dwavesys.com/sapi/"
solver = "Advantage_system4.1"
token = config.TOKEN
anealing_time = 1000
dw_sampler = DWaveSampler(solver=solver, endpoint=endpoint, token=token, properties={"annealing_time": anealing_time})
sampler = EmbeddingComposite(dw_sampler)

In [43]:
sampleset = sampler.sample_qubo(qubo, num_reads=1000,label = 'N-Queens')

## Result

In [44]:
#best solution
best_solution = sampleset.record[0]
print(best_solution.energy)

45.0


In [45]:
# Decode the solution
decoded_samples = model.decode_sampleset(sampleset, feed_dict=feed_dict)
for sample in decoded_samples:
    print(sample.constraints(only_broken=True))

{'col': (False, 23.0), 'num': (False, 9.0), 'diag': (False, 8.0), 'row': (False, 3.0)}
{'row': (False, 6.0), 'diag': (False, 16.0), 'num': (False, 16.0), 'col': (False, 18.0)}
{'col': (False, 16.0), 'num': (False, 16.0), 'diag': (False, 20.0), 'row': (False, 6.0)}
{'col': (False, 26.0), 'num': (False, 16.0), 'diag': (False, 12.0), 'row': (False, 4.0)}
{'row': (False, 4.0), 'diag': (False, 20.0), 'num': (False, 16.0), 'col': (False, 20.0)}
{'row': (False, 8.0), 'diag': (False, 16.0), 'num': (False, 16.0), 'col': (False, 22.0)}
{'row': (False, 4.0), 'diag': (False, 16.0), 'num': (False, 16.0), 'col': (False, 28.0)}
{'row': (False, 6.0), 'diag': (False, 16.0), 'num': (False, 16.0), 'col': (False, 26.0)}
{'col': (False, 20.0), 'num': (False, 16.0), 'diag': (False, 22.0), 'row': (False, 6.0)}
{'col': (False, 20.0), 'num': (False, 16.0), 'diag': (False, 26.0), 'row': (False, 6.0)}
{'col': (False, 22.0), 'num': (False, 16.0), 'diag': (False, 24.0), 'row': (False, 8.0)}
{'col': (False, 17.0), 

In [8]:
sampleset.to_pandas_dataframe().to_csv('nqueen_result.csv')

In [31]:
sampleset.to_pandas_dataframe().head()

,x[0][0],x[0][1],x[0][2],x[0][3],x[0][4],x[0][5],x[0][6],x[0][7],x[1][0],x[1][1],...,x[7][1],x[7][2],x[7][3],x[7][4],x[7][5],x[7][6],x[7][7],chain_break_fraction,energy,num_occurrences
0,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0.375000,-39.0,1
1,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0.375000,-39.0,1
2,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0.390625,-30.0,1
3,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0.359375,-28.0,1
4,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0.375000,-28.0,1


In [46]:
best = best_solution[0].reshape(N, N).T
print(best)

[[0 1 0 0 0 0 1 0]
 [1 1 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0]]
